In [1]:
from pydantic_ai import Agent, BinaryContent
from pydantic_ai.models.gemini import GeminiModel
from pydantic_ai.providers.google_gla import GoogleGLAProvider
from dotenv import load_dotenv
import os
import nest_asyncio
from IPython.display import display, Audio
import requests
from token_creator import get_creds
nest_asyncio.apply()
load_dotenv()
from cortana_agent import Cortana_agent
google_api_key=os.getenv('google_api_key')
tavily_key=os.getenv('tavily_key')
hf_token=os.getenv('hf_token')
pse=os.getenv('pse')
creds=get_creds()


In [2]:
api_keys={
    'google_api_key':google_api_key,
    'tavily_key':tavily_key,
    'pse':pse,
    'creds':creds
}

In [3]:
cortana=Cortana_agent(api_keys)

In [4]:
import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write as write_wav
import io

In [5]:
def record_audio(duration=3, sample_rate=44100, channels=1):
    """
    Record audio for a specified duration and return it as bytes.
    
    Args:
        duration (float): Recording duration in seconds
        sample_rate (int): Sample rate in Hz
        channels (int): Number of audio channels (1 for mono, 2 for stereo)
    
    Returns:
        bytes: Audio data in WAV format as bytes
    """
    print(f"Recording for {duration} seconds...")
    
    # Record audio
    recording = sd.rec(
        int(duration * sample_rate),
        samplerate=sample_rate,
        channels=channels,
        dtype='float32'
    )
    sd.wait()  # Wait until recording is finished
    print("Recording finished!")
    
    # Convert to int16 format (standard for WAV files)
    recording = (recording * 32767).astype(np.int16)
    
    # Save to bytes buffer
    buffer = io.BytesIO()
    write_wav(buffer, sample_rate, recording)
    audio_bytes = buffer.getvalue()
    buffer.close()
    return audio_bytes

In [8]:
audio=record_audio()
res=cortana.chat([BinaryContent(data=audio, media_type='audio/wav')])

API_URL = "https://router.huggingface.co/fal-ai/fal-ai/kokoro/american-english"
headers = {
    "Authorization": f"Bearer {hf_token}",
}

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

audio = query({
    "text": res})
# You can access the audio with IPython.display for example

audio=audio.get("audio").get('url')
audio=Audio(audio, autoplay=True)



Recording for 3 seconds...
Recording finished!


In [9]:
audio